In [183]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [184]:
import sys
from nplinker import NPLinker
from logconfig import LogConfig
from metabolomics import Spectrum
%reload_ext autoreload
%autoreload 2

In [185]:
# configuring NPLinker in a notebook env is now done either by passing in the name of a config file,
# or by passing in a dict which corresponds to the structure of the config file. Usually it will be
# easier to edit the file and simply pass the filename like this:
npl = NPLinker('latest_api_demo_sr.toml')

# the above step will attempt to discover the files to be loaded from the dataset and complain
# if they're not as expected. Next, actually load the data files
if not npl.load_data():
    raise Exception('Failed to load data')
 

10:34:02 [WARNING] loader.py:103, Optional file/directory "/Users/simon/TempStuff/carnegie_new/mibig_json" does not exist or is not readable!
Loaded 3107 molecules
10:34:05 [INFO] metabolomics.py:422, Loading annotations from GNPS database matches
Found 0 MiBIG json files
10:34:05 [WARNING] loader.py:134, Missing annotation/cluster tsv files in folder: /Users/simon/TempStuff/carnegie_new/bigscape/Saccharides
# MiBIG BGCs = 2383, non-MiBIG BGCS = 222, total bgcs = 2605, GCFs = 1655


In [186]:
# The scoring methods are defined and configured in the default configuration file at 
# ~/.config/nplinker/nplinker.toml, but will be overridden by the config file you loaded above,
# and the scoring methods can be easily changed once the NPLinker object has been created, e.g.:

# ensure only metcalf scoring is enabled, and set a 99% significance percentile threshold
print('Currently enabled scoring methods: {}'.format(npl.scoring.enabled()))
npl.scoring.likescore.enabled = False
# npl.scoring.likescore.cutoff = <scoring cutoff threshold>
npl.scoring.hg.enabled = False
# npl.scoring.hg.prob = <probability threshold>
npl.scoring.metcalf.enabled = True
npl.scoring.metcalf.sig_percentile = 99
print('Currently enabled scoring methods: {}'.format(npl.scoring.enabled()))

Currently enabled scoring methods: [namespace(enabled=True, name='metcalf', sig_percentile=99)]
Currently enabled scoring methods: [namespace(enabled=True, name='metcalf', sig_percentile=99)]


In [187]:
# to check if a spectrum has any of these can use .has_gnps_annotations method:
spectra_with_gnps_matches = [s for s in npl.spectra if s.has_gnps_annotations()]
print('found {} spectra'.format(len(spectra_with_gnps_matches)))

for s in spectra_with_gnps_matches:
    print(len(s.get_gnps_annotations()))

for gnps_anno in spectra_with_gnps_matches[0].get_gnps_annotations():
    # print as string
    print(gnps_anno)
    # access individual fields
    print(gnps_anno.id, gnps_anno.score, gnps_anno.name, gnps_anno.organism)
    # URLs for viewing spectrum info
    print(gnps_anno.png_url)
    print(gnps_anno.spec_url)

found 25 spectra
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
GNPSAnnotation(id=CCMSLIB00000081185, score=0.76303, name=cyclo-[L-(4-hydroxy-Pro)-L-leu], organism=GNPS-LIBRARY)
CCMSLIB00000081185 0.76303 cyclo-[L-(4-hydroxy-Pro)-L-leu] GNPS-LIBRARY
https://metabolomics-usi.ucsd.edu/png/?usi=mzspec:GNPSLIBRARY:CCMSLIB00000081185
https://metabolomics-usi.ucsd.edu/spectrum/?usi=mzspec:GNPSLIBRARY:CCMSLIB00000081185


In [188]:
# this step generates scores for all objects and enabled scoring methods, so it can be
# quite lengthy. The random_count parameter determines the number of randomised instances
# of Spectrum <=> Strain mappings that will be generated during the process.
if not npl.process_dataset(random_count=10):
    raise Exception('Failed to process dataset')
print('Completed generating scores')

Completed generating scores


In [189]:
# to get results once the scores are generated, first select an object you're interested 
# in, then call get_links with a specific scoring method. You can also pass a list of 
# objects as the first parameter. The method returns a list which contains only those
# objects that satisfy the scoring criteria (so here only those with a significance 
# percentile score of >= 99 as set above)
test_gcf = npl.gcfs[8]
results = npl.get_links(test_gcf, npl.scoring.metcalf)
if test_gcf not in results:
    print('No results found!')
else:
    print('Found results for {}!'.format(test_gcf))
    # to get the objects that scored highly against this GCF, use links_for_obj. By
    # default it will return all objects, the type_ parameter can be used to filter
    # by class, so here it will only return spectra
    test_gcf_links = npl.links_for_obj(test_gcf, npl.scoring.metcalf, type_=Spectrum)
    
    # print the objects and their scores, plus common strains
    for obj, score in test_gcf_links:
        print('{} : score {}'.format(obj, score))
        # returns a dict indexed by (Spectrum, GCF) tuples, with 
        # the values being lists of strain names shared between the two
        common_strains = npl.get_common_strains(test_gcf, obj)
        if len(common_strains) > 0:
            strain_names = list(common_strains.values())[0]
            print('   {} shared strains: {}'.format(len(strain_names), strain_names))
        else:
            print('   (no shared strains)')
            
    print('{} total links found'.format(len(test_gcf_links)))
        
    

10:34:15 [WARNING] nplinker.py:433, FIXME: MolecularFamily support
Found results for GCF(id=8, class=NRPS, gcf_id=285)!
Spectrum(id=2, spectrum_id=127) : score 26.0
   (no shared strains)
Spectrum(id=3, spectrum_id=135) : score 26.0
   (no shared strains)
Spectrum(id=6, spectrum_id=284) : score 26.0
   (no shared strains)
Spectrum(id=8, spectrum_id=328) : score 26.0
   (no shared strains)
Spectrum(id=10, spectrum_id=386) : score 26.0
   (no shared strains)
Spectrum(id=15, spectrum_id=546) : score 26.0
   (no shared strains)
Spectrum(id=43, spectrum_id=5572) : score 26.0
   (no shared strains)
Spectrum(id=50, spectrum_id=7693) : score 26.0
   (no shared strains)
Spectrum(id=55, spectrum_id=7701) : score 26.0
   (no shared strains)
Spectrum(id=56, spectrum_id=7703) : score 26.0
   (no shared strains)
Spectrum(id=59, spectrum_id=7706) : score 26.0
   (no shared strains)
Spectrum(id=61, spectrum_id=7709) : score 26.0
   (no shared strains)
Spectrum(id=72, spectrum_id=7723) : score 26.0
   

## Rosetta-stone linking

This is an example of how we would do the linking based upon Grimur's magic dictionary

firstly, make the spectral library object

this makes use of code in my molnet repository

The following is quite slow, and the SpecLib object could be pickled up and loaded in

In [190]:
sys.path.append('/Users/simon/git/molnet/code')
# the following file can be found on uist at /srv/data/mibig-links/20190805/matched_mibig_gnps_update.mgf
mgf_file = '/Users/simon/git/molnet/lib/matched_mibig_gnps_update.mgf'
from spec_lib import SpecLib
s = SpecLib(mgf_file)
s._load_mgf()
s.filter()

Loaded 100 spectra
Loaded 200 spectra
Loaded 300 spectra
Loaded 400 spectra
Loaded 500 spectra
Loaded 600 spectra
Loaded 700 spectra
Loaded 800 spectra
Loaded 900 spectra
Loaded 1000 spectra
Loaded 1100 spectra
Loaded 1200 spectra
Loaded 1300 spectra
Loaded 1400 spectra
Loaded 1500 spectra
Loaded 1600 spectra
Loaded 1700 spectra
Loaded 1800 spectra
Loaded 1900 spectra
Loaded 2000 spectra
Filtered 100
Filtered 200
Filtered 300
Filtered 400
Filtered 500
Filtered 600
Filtered 700
Filtered 800
Filtered 900
Filtered 1000
Filtered 1100
Filtered 1200
Filtered 1300
Filtered 1400
Filtered 1500
Filtered 1600
Filtered 1700
Filtered 1800
Filtered 1900
Filtered 2000


Now do the spectral matching from the spectrum objects nplinker has loaded

Note to simon: might want to set the ms1_tol parameter to spectral match high to find analogues

In [191]:
spec_hits = {}
for i,sp in enumerate(npl.spectra):
    if not hasattr(sp, 'normalised_peaks'):
        # have to have a normalised peaks
        sp.normalised_peaks = sqrt_normalise(sp.peaks)
    hits = s.spectral_match(sp,ms1_tol = 100,score_thresh=0.5)
    if len(hits) > 0:
        spec_hits[sp] = hits
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


Spectral matching needs normalised peaks -- not sure where it's best to put this...

In [192]:
import math
def sqrt_normalise(peaks):
    temp = []
    total = 0.0
    for mz,intensity in peaks:
        temp.append((mz,math.sqrt(intensity)))
        total += intensity
    norm_facc = math.sqrt(total)
    normalised_peaks = []
    for mz,intensity in temp:
        normalised_peaks.append((mz,intensity/norm_facc))
    return normalised_peaks

In [200]:
print(len(spec_hits))

238


## load the rosetta stone

this file is available from /srv/data/mibig-links/20190805/matched_mibig_gnps_update.csv

In [194]:
import csv
rosetta_file = '/Users/simon/git/molnet/lib/matched_mibig_gnps_update.csv'

gnps2mibig = {}
mibig2gnps = {}

with open(rosetta_file,'r') as f:
    reader = csv.reader(f)
    heads = next(reader)
    for line in reader:
        gnps = line[0]
        mibig = line[3]
        if not gnps in gnps2mibig:
            gnps2mibig[gnps] = [mibig]
        else:
            gnps2mibig[gnps].append(mibig)
        if not mibig in mibig2gnps:
            mibig2gnps[mibig] = [gnps]
        else:
            mibig2gnps[mibig].append(gnps)

In [214]:
def parse_kcb(kcb_file):
    with open(kcb_file,'r') as f:
        line = next(f)
        while not line.startswith('Table of genes'):
            line = next(f)
        # now we're in the first block
        top_block = []
        while True:
            line = next(f)
            if line.startswith('Significant'):
                break
            else:
                if len(line) > 1:
                    top_block.append(line.rstrip())
        # now we're in the second block
        second_block = []
        while True:
            line = next(f)
            if line.startswith('Details'):
                break
            else:
                if len(line) > 1:
                    second_block.append(line.rstrip())
        while True:
            try:
                line = next(f)
                if line.startswith('>>'):
                    break
            except:
                return None
        details = []
        finished = False
        while not finished:
            temp_list = []
            while True:
                try:
                    line = next(f)
                    if line.startswith('>>'):
                        # finished one
                        details.append(temp_list)
                        break
                    else:
                        if len(line) > 1:
                            temp_list.append(line.rstrip())
                except:
                    details.append(temp_list)
                    finished = True
                    break
        # do some processing on the blocks
        # firstly, extract the genes from the BGC -- stored in the first block
        bgc_genes = set()
        for line in top_block:
            tokens = line.split()
            bgc_genes.add(tokens[0])
        # secondly, extract the BGCs that are mentioned here
        mibig_bgcs = []
        for line in second_block:
            tokens = line.split()
            bgc_id = tokens[1]
            bgc_product_name = tokens[2]
            mibig_bgcs.append((bgc_id,bgc_product_name))
        hits = {}
        for i,detail in enumerate(details):
            current_bgc_id = detail[0].split()[1]
            hits[current_bgc_id] = []
            assert current_bgc_id == mibig_bgcs[i][0] # they should be in the same order
            table_pos = detail.index('Table of genes, locations, strands and annotations of subject cluster:')
            pos = detail.index('Table of Blast hits (query gene, subject gene, %identity, blast score, %coverage, e-value):')
            print(detail[table_pos:pos])
            for line in detail[pos+1:]:
                tokens = line.split()
                bgc_id = tokens[0]
                hits[current_bgc_id].append({'source_bgc_gene':tokens[0],'mibig_bgc_gene':tokens[1],'identity_percent':int(tokens[2]),'blast_score':int(tokens[3]),'all_bgc_genes':bgc_genes})
    return hits

In [215]:
bgc_hits = {}
for bgc in npl.bgcs:
    bgc_file = bgc.antismash_file
    if bgc_file:
        base_path = os.sep.join(bgc_file.split(os.sep)[:-1])
        base_path = os.path.join(base_path,'knownclusterblast')
        genbank_file = bgc_file.split(os.sep)[-1]
        # remove the regionXXX and turn it into _cXXX
        tokens = genbank_file.split('region')
        number = int(tokens[1].split('.')[0])
        start_name = tokens[0][:-1] # remove the last dot
        start_name += '_c{}.txt'.format(number)
        kcb_name = os.path.join(base_path,start_name)
        hits = parse_kcb(kcb_name)
        if hits:
            bgc_hits[bgc] = hits
print("{} BGCs have one or more known cluster blast hits".format(len(bgc_hits)))

['Table of genes, locations, strands and annotations of subject cluster:', 'AAD44199.1\tAAD44199.1\t47\t1132\t+\tunknown', 'AAD44200.1\tAAD44200.1\t1524\t2441\t-\tIS1601-D', 'AAD44201.1\tAAD44201.1\t2441\t2785\t-\tIS1601-C', 'AAD44202.1\tAAD44202.1\t2837\t4039\t-\tIS1601-B', 'AAD44203.1\tAAD44203.1\t4298\t5545\t-\tIS1601-A', 'AAD44204.1\tAAD44204.1\t5791\t6876\t+\tA-protein-like_protein', 'AAD44205.1\tAAD44205.1\t6996\t8057\t+\tGepiA', 'AAD44206.1\tAAD44206.1\t8321\t9448\t+\tMtfA', 'AAD44207.1\tAAD44207.1\t9715\t10536\t+\tMtfB', 'AAD44208.1\tAAD44208.1\t10658\t11935\t+\tGtfA', 'AAD44209.1\tAAD44209.1\t12339\t13625\t+\tRtfA', 'AAD44210.1\tAAD44210.1\t13727\t14527\t+\tMtfC', 'AAD44211.1\tAAD44211.1\t14627\t15445\t+\tmtfD', 'AAD44212.1\tAAD44212.1\t16004\t17236\t-\ttransposase', 'AAD44213.2\tAAD44213.2\t17728\t18984\t-\tGtfB', 'AAD44214.2\tAAD44214.2\t19126\t19779\t+\tunknown', 'AAD44215.1\tAAD44215.1\t20437\t20781\t+\ttransposase', 'AAD44216.1\tAAD44216.1\t20781\t21698\t+\ttransposase', 

In [197]:
# make a reverse dictionary
mibig2bgc = {}
for bgc in bgc_hits:
    for mibig_bgc_id in bgc_hits[bgc]:
        if not mibig_bgc_id in mibig2bgc:
            mibig2bgc[mibig_bgc_id] = set()
        mibig2bgc[mibig_bgc_id].add(bgc)

In [198]:
rosetta_hits = []
for spec in spec_hits:
    for gnps_id,score in spec_hits[spec]:
        for mibig_id in gnps2mibig[gnps_id]:
            print(mibig_id)
            if mibig_id in mibig2bgc:
                for bgc in mibig2bgc[mibig_id]:
                    rosetta_hits.append((spec,gnps_id,mibig_id,bgc))
print("Found {} rosetta hits".format(len(rosetta_hits)))

BGC0000088
BGC0000089
BGC0000098
BGC0000088
BGC0000089
BGC0000098
BGC0001238
BGC0001239
BGC0001238
BGC0001239
BGC0001238
BGC0001239
BGC0001238
BGC0001239
BGC0001238
BGC0001239
BGC0001238
BGC0001239
BGC0001238
BGC0001239
BGC0001238
BGC0001239
BGC0001238
BGC0001239
BGC0000182
BGC0000182
BGC0000088
BGC0000089
BGC0000098
BGC0000088
BGC0000089
BGC0000098
BGC0000895
BGC0000895
BGC0000894
BGC0001338
BGC0000070
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0001735
BGC0000182
BGC0000280
BGC0000281
BGC0000088
BGC0000089
BGC0000098
BGC0000088
BGC0000089
BGC0000098

In [199]:
for hit in rosetta_hits:
    print(hit[0],"<-->",hit[3]," via (",hit[1],",",hit[2],")")

Spectrum(id=353, spectrum_id=66209) <--> BGC(name=KRD197_Scaffold_16.region002, strain=KRD197)  via ( CCMSLIB00000565239 , BGC0000893 )
Spectrum(id=353, spectrum_id=66209) <--> BGC(name=KRD162_Scaffold_3.region002, strain=KRD162)  via ( CCMSLIB00000565239 , BGC0000893 )
Spectrum(id=353, spectrum_id=66209) <--> BGC(name=KRD175_Scaffold_2.region003, strain=KRD175)  via ( CCMSLIB00000565239 , BGC0000893 )
Spectrum(id=353, spectrum_id=66209) <--> BGC(name=KRD162_Scaffold_15.region002, strain=KRD162)  via ( CCMSLIB00000565239 , BGC0000893 )
Spectrum(id=358, spectrum_id=66494) <--> BGC(name=KRD197_Scaffold_4.region001, strain=KRD197)  via ( CCMSLIB00000569825 , BGC0000209 )
Spectrum(id=439, spectrum_id=69424) <--> BGC(name=KRD197_Scaffold_16.region002, strain=KRD197)  via ( CCMSLIB00000567446 , BGC0000893 )
Spectrum(id=439, spectrum_id=69424) <--> BGC(name=KRD162_Scaffold_3.region002, strain=KRD162)  via ( CCMSLIB00000567446 , BGC0000893 )
Spectrum(id=439, spectrum_id=69424) <--> BGC(name=KR

Spectrum(id=1733, spectrum_id=149254) <--> BGC(name=KRD171_Scaffold_13.region001, strain=KRD171)  via ( CCMSLIB00000218169 , BGC0000858 )
Spectrum(id=1733, spectrum_id=149254) <--> BGC(name=KRD197_Scaffold_1.region001, strain=KRD197)  via ( CCMSLIB00000218169 , BGC0000858 )
Spectrum(id=1733, spectrum_id=149254) <--> BGC(name=KRD202_Scaffold_6.region001, strain=KRD202)  via ( CCMSLIB00000218169 , BGC0000858 )
Spectrum(id=1733, spectrum_id=149254) <--> BGC(name=KRD168_c00062_NODE_62...region001, strain=KRD168)  via ( CCMSLIB00000218169 , BGC0000858 )
Spectrum(id=1733, spectrum_id=149254) <--> BGC(name=KRD172_Scaffold_2.region001, strain=KRD172)  via ( CCMSLIB00000218169 , BGC0000858 )
Spectrum(id=1733, spectrum_id=149254) <--> BGC(name=KRD209_Scaffold_24.region001, strain=KRD209)  via ( CCMSLIB00000218169 , BGC0000858 )
Spectrum(id=1733, spectrum_id=149254) <--> BGC(name=KRD171_Scaffold_13.region001, strain=KRD171)  via ( CCMSLIB00000218169 , BGC0000859 )
Spectrum(id=1733, spectrum_id=14

## Todo:

- At the moment we get lots of hits per GNPS,MiBIG pair because they are in lots of BGCs
- We also should percolate the scores (both of the spectral match and the knownclusterblast) to the output
- Parameterise (at least) two parameters in the spectral matching: score threshold and ms1_tol. At the moment, MS1_tol will only find things with near identical MS1 m/z, which precludes analogues.
- The code for getting the knownclusterblast name and parsing the knownclusterblast file is horrific... :-)

In [249]:
def process_bgc_hit(hit):
    # process the hit to compress it into more useful info
    # computes the total identity score for each mibig entry
    # and divides by the number of mibig genes
    # i.e. the score represents how much of the mibig is 
    # reflected in the source bgc
    mibig_bgcs = list(hit.keys())
    scores = {}
    for mibig_id in mibig_bgcs:
        n_source_genes = len(hit[mibig_id][0]['all_bgc_genes'])
        n_mibig_genes = len(hit[mibig_id][0]['all_mibig_genes'])
        total_hit_identity = 0
        for hit_gene in hit[mibig_id]:
            identity_percent = hit_gene['identity_percent']
            total_hit_identity += identity_percent / 100.0
        score = total_hit_identity / n_mibig_genes
        scores[mibig_id] = score
    return scores
bgc_hit_summary_scores = {}
for bgc,hit in bgc_hits.items():
    bgc_hit_summary_scores[bgc] = process_bgc_hit(hit)
# so, in bgc_hit_summary scores, we have the 